In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
import os
from librosa import display,util,load,feature

In [14]:
#media_dir_path  = "C:/Users/rujut/Desktop/Sleigh Group/Australia/Australia/Adrienne O'Connor"

media_dir_path  = "//Mac/Home/Downloads/VoiceSamples/Manila/Elle Genovata"

print(media_dir_path)

//Mac/Home/Downloads/VoiceSamples/Manila/Elle Genovata


In [3]:
media_files = listdir(media_dir_path)
print(media_files)

['Raine Natural.wav']


In [4]:
frequency_range_file = 'FrequencyRange.csv'
freq_ref= pd.read_csv(frequency_range_file)
freq_ref

,FrequencyRange,Low,High
0,Sub-bass,20,60
1,Bass,60,250
2,Low midrange,250,500
3,Midrange,500,2000
4,Upper midrange,2000,4000
5,Prescent,4000,6000
6,Brilliance,6000,20000


In [5]:
def _lookup_req_range(mean_cent, max_cent, freq_ref):
    mean_range_desc = ''
    max_range_desc = ''

    for i in range(0,len(freq_ref)):
        if i == 0:
            if freq_ref.iloc[i]['High'].astype(float) > mean_cent:
                mean_range_desc = freq_ref.iloc[i]['FrequencyRange']
        elif i == len(freq_ref) - 1:
            if freq_ref.iloc[i]['Low'].astype(float) <= max_cent:     
                max_range_desc = freq_ref.iloc[i]['FrequencyRange']
        else:
            if freq_ref.iloc[i]['Low'].astype(float) <= mean_cent and freq_ref.iloc[i]['High'].astype(float) > mean_cent:
                mean_range_desc = freq_ref.iloc[i]['FrequencyRange']
    
    for j in range(0,len(freq_ref)):
        if j == 0: 
            if freq_ref.iloc[j]['High'].astype(float) > mean_cent:
                max_range_desc = freq_ref.iloc[i]['FrequencyRange']
        elif j== len(freq_ref) - 1:
            if freq_ref.iloc[j]['Low'].astype(float) <= max_cent:     
                max_range_desc = freq_ref.iloc[j]['FrequencyRange']
        else:
            if freq_ref.iloc[j]['Low'].astype(float) <= max_cent and freq_ref.iloc[j]['High'].astype(float) > max_cent:
                max_range_desc = freq_ref.iloc[j]['FrequencyRange']
    
    freq_range_list = [mean_range_desc, max_range_desc]

    return freq_range_list

In [6]:
def _find_features(media_dir_path, freq_ref):
    files = util.find_files(media_dir_path, ext=['wav'])
    
    # Create properties dataframe
    col_name = ['file_name','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc','spec_cent_max']
    sample_dataset = pd.DataFrame(columns = col_name)
    
    for y in files:
        librosa_load, librosa_sampling_rate = load(y) 
        stft = feature.chroma_stft(y=librosa_load, sr=librosa_sampling_rate)
        spec_cent = feature.spectral_centroid(y=librosa_load, sr=librosa_sampling_rate)
        spec_bw = feature.spectral_bandwidth(y=librosa_load, sr=librosa_sampling_rate)
        rolloff = feature.spectral_rolloff(y=librosa_load, sr=librosa_sampling_rate)
        zcr = feature.zero_crossing_rate(librosa_load)
        mfcc = feature.mfcc(y=librosa_load, sr=librosa_sampling_rate)
      
        chroma_stft_mean = np.mean(stft)
        spec_cent_mean = np.mean(spec_cent)
        spec_bw_mean = np.mean(spec_bw)
        rolloff_mean = np.mean(rolloff)
        zcr_mean = np.mean(zcr)
        mfcc_mean = np.mean(mfcc)
        spec_cent_max = np.max(spec_cent)
        
        feature_dict = { 'file_name': y,
                         'chroma_stft': chroma_stft_mean, 
                         'spec_cent': spec_cent_mean, 
                         'spec_bw': spec_bw_mean,
                         'rolloff': rolloff_mean,
                         'zcr': zcr_mean,
                         'mfcc': mfcc_mean,
                         'spec_cent_max':spec_cent_max
                        }
 
        sample_dataset = sample_dataset.append(feature_dict, ignore_index = True)
    
    sample_dataset.reset_index(inplace=True, drop = True)
    mean_cent = sample_dataset["spec_cent"].mean()
    max_cent = sample_dataset["spec_cent_max"].max()
    range_list = _lookup_req_range(mean_cent, max_cent, freq_ref)
    
    return range_list,sample_dataset

In [15]:
range_list, sample_dataset = _find_features(media_dir_path,freq_ref)
range_list

['Upper midrange', 'Brilliance']

In [16]:
sample_dataset

,file_name,chroma_stft,spec_cent,spec_bw,rolloff,zcr,mfcc,spec_cent_max
0,\\Mac\Home\Downloads\VoiceSamples\Manila\Elle ...,0.287064,2016.209826,2115.573854,3735.152535,0.086932,-19.76239,8269.825002
